In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from sklearn.metrics import roc_auc_score
from keras.models import Sequential , Model
from keras.layers import Input , Dense , concatenate

Using TensorFlow backend.


In [3]:
from sklearn.model_selection import train_test_split

In [4]:
df_ad = pd.read_csv('../datadays2020/ad.csv')
df_train = pd.read_csv('../datadays2020/click_train.csv')
df_event = pd.read_csv('../datadays2020/event.csv')
df_page = pd.read_csv('../datadays2020/page.csv')
df_topic = pd.read_csv('../datadays2020/page_topic.csv')
df_test = pd.read_csv('../datadays2020/click_test.csv')
#df_upv = pd.read_csv('data/user_page_view.csv')


df = pd.merge(df_train , df_event , on='displayId' , how='left')
dft = pd.merge(df_test , df_event , on='displayId' , how='left')
t=min(df_event['timestamp'])
timetable = [(0,6), (6,12),(12,18),(18,24)]
def discretiseTime(t,timetable):
    t=t%24
    for i , e in enumerate(timetable):
        #print(t , e)
        if t>=e[0] and t<= e[1]:
            return i

df['timestamp'] = df['timestamp'].apply(lambda x : discretiseTime((x-t)/(1000*3600) , timetable)) 
dft['timestamp'] = dft['timestamp'].apply(lambda x : discretiseTime((x-t)/(1000*3600) , timetable))                                                 

df = pd.merge(df , df_page , on='pageId' , how='left')
dft = pd.merge(dft , df_page , on='pageId' , how='left')
df_topic=df_topic.sort_values(['pageId' , 'confidence'] , ascending=False)
df_topic=df_topic.drop_duplicates(['pageId'])
df = pd.merge(df , df_topic , on='pageId' , how='left')
dft = pd.merge(dft , df_topic , on='pageId' , how='left')
df = pd.merge(df , df_ad , on='adId' , how='left')
dft = pd.merge(dft , df_ad , on='adId' , how='left')

df['topicId'] = df['topicId'].fillna(0)
df['confidence'] = df['confidence'].fillna(0)
dft['topicId'] = dft['topicId'].fillna(0)
dft['confidence'] = dft['confidence'].fillna(0)

df['clicked'] = df['clicked'].astype(np.int8)
df['timestamp'] = df['timestamp'].astype(np.int8)
df['pageId'] = df['pageId'].astype(np.int32)
df['widgetId'] = df['widgetId'].astype(np.int16)
df['userId'] = df['userId'].astype(np.int32)
df['device'] = df['device'].astype(np.int8)
df['OS'] = df['OS'].astype(np.int8)
df['browser'] = df['browser'].astype(np.int8)
df['website'] = df['website'].astype(np.int32)
df['publisher'] = df['publisher'].astype(np.int16)
df['topicId']= df['topicId'].astype(np.int8)
df['confidence'] =df['confidence'].astype(np.float32)
df['campaignId'] = df['campaignId'].astype(np.int16)
df['advertiserId'] = df['advertiserId'].astype(np.int16)

#dft['clicked'] = dft['clicked'].astype(np.int8)
dft['timestamp'] = dft['timestamp'].astype(np.int8)
dft['pageId'] = dft['pageId'].astype(np.int32)
dft['widgetId'] = dft['widgetId'].astype(np.int16)
dft['userId'] = dft['userId'].astype(np.int32)
dft['device'] = dft['device'].astype(np.int8)
dft['OS'] = dft['OS'].astype(np.int8)
dft['browser'] = dft['browser'].astype(np.int8)
dft['website'] = dft['website'].astype(np.int32)
dft['publisher'] = dft['publisher'].astype(np.int16)
dft['topicId']= dft['topicId'].astype(np.int8)
dft['confidence'] =dft['confidence'].astype(np.float32)
dft['campaignId'] = dft['campaignId'].astype(np.int16)
dft['advertiserId'] = dft['advertiserId'].astype(np.int16)

df.to_csv('../temp/trainmerge.csv' , index=False)
dft.to_csv('../temp/testmerge.csv' , index=False)


In [5]:

features = ['adId' , 'timestamp' , 'topicId' ,'device','OS','browser','publisher', 'campaignId' , 'advertiserId' , 'widgetId' , 'website' ]

dftemp = df[features]
dfttemp = dft[features]
dftemp.head()



,adId,timestamp,topicId,device,OS,browser,publisher,campaignId,advertiserId,widgetId,website
0,7182,2,20,0,0,1,49,3402,7579,9519,477
1,7125,2,20,0,0,1,49,251,5971,9519,477
2,7181,2,20,0,0,1,49,3402,7579,9519,477
3,535,2,20,0,0,1,49,3186,1419,9519,477
4,7174,2,20,0,0,1,49,471,5528,9519,477


In [6]:
X_train , X_test , Y_train , Y_test = train_test_split(dftemp , df['clicked'] , test_size = 0.33 , random_state = 42)

In [7]:
X_train.head()

,adId,timestamp,topicId,device,OS,browser,publisher,campaignId,advertiserId,widgetId,website
1389672,6371,1,28,0,0,5,83,7967,1386,13463,88
2973315,5091,0,0,0,0,1,247,4047,9588,11656,280
3154462,2237,1,12,0,0,0,14,552,8843,607,15
891402,4395,1,28,0,0,0,341,7556,2186,9550,400
1932622,4389,2,4,0,0,0,9,7556,2186,13301,10


In [8]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer 
transformer = ColumnTransformer(  
    transformers=[
        ("all",        # Just a name
         OneHotEncoder(dtype = np.int8), # The transformer class
         [0,1,2,3,4,5,6,7,8,9,10]           # The column(s) to be applied on.
         ),
    ], remainder='passthrough'
)

transformer.fit(dftemp.append(dfttemp))

c:\users\sajjad\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


ColumnTransformer(n_jobs=None, remainder='passthrough', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('all', OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.int8'>, handle_unknown='error', n_values=None,
       sparse=True), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])])

In [9]:
n = int(0.66*len(df))
tplus = df[df['clicked']>0].sample(n=n//2 , replace=True)
tminus = df[df['clicked']==0].sample(n=n//2 , replace=True)
X_train = pd.concat([tplus ,tminus])
X_train = X_train.sample(frac=1.00)
Y_train = X_train['clicked']
X_train = X_train[features]

In [10]:
data = transformer.transform(X_train)
data.shape

(2403880, 9027)

In [12]:
#wide
wide = Input(shape=(data.shape[1],))

#deep
deep_data = Input(shape=(data.shape[1],))
deep = Dense(input_dim=data.shape[1] , output_dim=256 , activation='relu')(deep_data)
deep = Dense(128 , activation = 'relu')(deep)

#wide & deep
wide_deep = concatenate([wide , deep])
wide_deep = Dense(1 , activation='sigmoid')(wide_deep)
model = Model(inputs=[wide , deep_data] , outputs=wide_deep)

model.compile(
    optimizer = 'rmsprop' ,
    loss = 'binary_crossentropy' ,
    metrics = ['accuracy']
)


c:\users\sajjad\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=9027, activation="relu", units=256)`
  


In [13]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 9027)         0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 256)          2311168     input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 9027)         0                                            
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 128)          32896       dense_1[0][0]                    
____________________________________________________________________________________________

In [14]:
from keras.models import load_model
model = load_model('../my_model.h5')

In [15]:
model.fit([data , data] , Y_train , epochs=5 , batch_size=512)

Epoch 1/5
 543744/2403880 [=====>........................] - ETA: 4:59 - loss: 0.6368 - accuracy: 0.6495

KeyboardInterrupt: 

In [14]:
model.fit([X_train , X_train] , Y_train , epochs=20, batch_size=32)

Epoch 1/20
2403880/2403880 [==============================] - 85s 36us/step - loss: 5.4434 - accuracy: 0.5035
Epoch 2/20
2403880/2403880 [==============================] - 85s 35us/step - loss: 5.4319 - accuracy: 0.50380s - loss: 5.4315 -  - ETA: 0s - loss: 5
Epoch 3/20
2403880/2403880 [==============================] - 84s 35us/step - loss: 5.4388 - accuracy: 0.5031
Epoch 4/20
2403880/2403880 [==============================] - ETA: 0s - loss: 5.4319 - accuracy: 0.5037 ETA: 0s - loss: 5.4324 - ac - 85s 36us/step - loss: 5.4319 - accuracy: 0.5037
Epoch 5/20
2403880/2403880 [==============================] - 85s 35us/step - loss: 5.4353 - accuracy: 0.5038
Epoch 6/20
2403880/2403880 [==============================] - 84s 35us/step - loss: 5.4305 - accuracy: 0.5038
Epoch 7/20
2403880/2403880 [==============================] - 87s 36us/step - loss: 5.4323 - accuracy: 0.5041
Epoch 8/20
2403880/2403880 [==============================] - 86s 36us/step - loss: 5.4430 - accuracy: 0.5032
Epoch 9/

KeyboardInterrupt: 

In [45]:
import tensorflow as tf
from tffm import TFFMClassifier , utils
order = 2
model = TFFMClassifier(
    order=order, 
    rank=40, 
    optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
    n_epochs=4, 
    batch_size=512,
    init_std=0.001,
    reg=0.001,
    input_type='sparse',
    seed=42
)
model.fit(data, Y_train.values, show_progress=True)
predictions = model.predict(data)
print('[order={}] accuracy: {}'.format(order, accuracy_score(Y_train.values, predictions)))
predictions = model.predict(datatest)
print('[order={}] accuracy: {}'.format(order, accuracy_score(Y_test.values, predictions)))
#model.destroy()



  0%|          | 0/4 [00:00<?, ?epoch/s]

 25%|██▌       | 1/4 [09:28<28:25, 568.61s/epoch]

 50%|█████     | 2/4 [18:59<18:58, 569.27s/epoch]

 75%|███████▌  | 3/4 [28:17<09:25, 565.79s/epoch]

100%|██████████| 4/4 [37:38<00:00, 564.49s/epoch]

[order=2] accuracy: 0.8030667088207398
[order=2] accuracy: 0.69531116751987


In [30]:
from sklearn.metrics import roc_auc_score, accuracy_score
predictions = model.predict(data)
print('[order={}] accuracy: {}'.format(order, accuracy_score(Y_train.values, predictions)))
predictions = model.predict(datatest)
print('[order={}] accuracy: {}'.format(order, accuracy_score(Y_test.values, predictions)))

[order=2] accuracy: 0.9552810456428773


NameError: name 'datatest' is not defined

In [44]:
datatest = transformer.transform(X_test)
datatest.shape 

(1201941, 197444)

In [34]:
predictions = model.predict(datatest)
print('[order={}] accuracy: {}'.format(order, accuracy_score(Y_test.values, predictions)))

[order=2] accuracy: 0.7733757314210931


In [35]:
len(Y_test[Y_test==0])/len(Y_test)

0.776649602601126

In [46]:
Xtest = dft
datates = transformer.transform(dfttemp)
Xtest['rank'] = model.predict_proba(datates)[:,1]
diad={}
for i,r in Xtest.iterrows():
    if r['displayId'] in diad:
        diad[r['displayId']].append(i)
    else:
        diad[r['displayId']] = [i,]
for i in diad:
    diad[i] = sorted(diad[i] , key =lambda x: -1 * Xtest.iloc[x]['rank'] )
arr = np.zeros(len(Xtest))

for i,r in Xtest.iterrows():
    arr[i] = diad[r['displayId']].index(i) + 1
Xtest['ranked'] = arr
#dftest.to_csv('../submit/sub4.csv',header=None,index=False)
dfsub = Xtest[['displayId' , 'adId' ,'ranked']]
dfsub['ranked'] = Xtest['ranked'].apply(lambda x : int(x) )
dfsub.head()
dfsub.to_csv('Sub26.csv',header=None,index=False)

c:\users\sajjad\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [20]:
len(dft['displayId'].unique())

270768